### Aliasing
**Learning objectives: using fast fourier transforms, aliasing.**

A noiseless analogue signal can be recovered perfectly from discretely sampled data, provided the sampling rate is greater than double the highest frequency component. This is Nyquist's theorem. If a signal is sampled below this rate, aliasing occurs.

In [8]:
import numpy as np
from numpy import pi
from numpy.fft import fft, ifft, fftshift

from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook, gridplot
from ipywidgets import widgets, interact
from IPython.display import display

output_notebook()

sample_no = 256
signal_frequency = 100.0

#time domain function f(t)
t = np.linspace(-0.5,0.5,sample_no)
f = np.cos(2*signal_frequency*pi*t)

Loading BokehJS ...

Time is plotted from 0s to 1s and there are 256 samples. The sampling frequency is therefore 256 Hz, giving a Nyqyuist limit of 128 Hz. Any signal of frequency greater than 128 Hz will be aliased to a frequency below this.

In [9]:
#frequency space domain function F(v) = FT(f(t))
v = np.linspace(-0.5*sample_no, 0.5*sample_no, sample_no)
F = fftshift(abs(fft(f)))

#plot the function F(v)
p1 = figure(x_axis_label = 'Frequency/Hz', y_axis_label = 'FT(cosine)')
q1 = p1.line(v, F)

show(p1)

In [10]:
def change_frequency(signal_frequency):
    f = np.cos(2*signal_frequency*pi*t)
    F = fftshift(abs(fft(f)))
    q1.data_source.data['y'] = F
    push_notebook()

interact(change_frequency, signal_frequency=widgets.IntSlider(min=0,max=256,step=1,value=8));

When the signal frequency exceeds the Nyquist limit it is aliased back to a lower frequency. This can lead to severe distortion of sampled signals. A frequency $f$ in the signal with $f_{sampling} < 2f < 2f_{sampling}$ will 'bounce back' and appear at $2f_{sampling} - f$. 

Consider aliasing of a pair of sinc functions:

In [4]:
#the sinc frequency is taken to mean the frequency of the sine
sinc_frequency = 10.0

#time domain signal
double_sinc = np.sinc(2*sinc_frequency*pi*t) - 0.5*np.sinc(2*0.5*sinc_frequency*pi*t)

p2 = figure(height = 300, width = 300, title = 'double_sinc', x_axis_label = 'time/s')
p2.title.text_font_size = '10pt'
q2 = p2.line(t, double_sinc)

In [5]:
#frequency space signal
F_double_sinc = fftshift(fft(double_sinc))

p3 = figure(height = 300, width = 300, title = 'F_double_sinc = FT{double_sinc}', x_axis_label = 'frequency/Hz')
p3.title.text_font_size = '10pt'
q3 = p3.line(v, abs(F_double_sinc))

In [6]:
#inverse fourier transform back
I_double_sinc = ifft(F_double_sinc)

p4 = figure(height = 300, width = 300, title = 'I_double_sinc = IFT{F_double_sinc}', x_axis_label = 'time/s')
p4.title.text_font_size = '10pt'
q4 = p4.line(t, np.real(I_double_sinc))

p5 = gridplot([[p2,p3,p4]])
show(p5)

In [7]:
def change_sinc_frequency(sinc_frequency):
    double_sinc = np.sinc(2*sinc_frequency*pi*t) - 0.5*np.sinc(2*0.5*sinc_frequency*pi*t)
    F_double_sinc = fftshift(fft(double_sinc))
    I_double_sinc = ifft(F_double_sinc)
    #update the data on the graphs
    q2.data_source.data['y'] = double_sinc
    q3.data_source.data['y'] = abs(F_double_sinc)
    q4.data_source.data['y'] = np.real(I_double_sinc)
    #push this change to the notebook
    push_notebook()

#create an interactive slider to control plot properties
interact(change_sinc_frequency, sinc_frequency=widgets.IntSlider(min=0,max=256,step=1,value=8));

Even at low frequencies, the recovered `I_double_sinc` function appears to have little in common with the original `double_sinc`. This is because the fourier transform `F_double_sinc` is truncated by the finite length of the array, 256 in this case. This has the same effect as multiplying the fourier transform by a top hat of width 256 Hz, centred on 0 Hz.

Consequently, `I_double_sinc` is the original `double_sinc` convolved with a sinc function of high frequency.
Q: What is this frequency?

A striking feature of the time domain plots (left and right) is their change in shape as aliasing sets in. When the higher frequency components of the signal are aliased, but the lower are not, the range of frequencies in the signal decreases. This is observed as a narrowing of the top hats in the central plot.
There exists a frequency at which the top hats become dirac-delta functions and the `double_sinc` function become purely sinusoidal.